## Analyzing the Impact of TELs on Debt Issues

This Notebook uses the data constructed in [sas2csv](https://github.com/choct155/TELs_debt/blob/master/code/sas2csv.ipynb) and [DebtDataSeries](https://github.com/choct155/TELs_debt/blob/master/code/DebtDataSeries.ipynb) to evaluate the impact of tax and expenditure limitations on debt issues by county.  This Notebook will do the following:

1. Subset to the variables critical to our analysis (**Data Input**);
2. Build specifications that feature a set of debt related dependent variables (**Model Design**);
3. Estimate the relationship between TELs and debt by way of pooled and fixed effect models (**Estimation**).

In [226]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sb
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas.io.data as web

%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## Data Input

Our data is housed in ... the **`data/`** directory.  We are looking for `debt_out.csv` which has aggregate debt issue, institutional, socioeconomic, and spatial information aggregated to the county level.

In [227]:
!ls -l ../data/

total 376516
-rw-r--r-- 1 root root    165888 Nov 12 06:20 13slsstab1a.xls
-rw-r--r-- 1 root root     93112 Nov 12 06:20 2013_GFS_debt.xcf
-rw-r--r-- 1 root root  12226235 Nov 12 06:20 bonds.csv
-rwxr-xr-x 1 root root  77725960 Nov 13 18:49 costat_mod_vars1940_2010.csv
-rwxr-xr-x 1 root root   2966812 Nov 13 19:58 cty_coverage.csv
-rw-r--r-- 1 root root         0 Nov 12 06:20 current_issue_geocode_list.csv
-rwxr-xr-x 1 root root  58016001 Nov 13 19:58 debt_out.csv
-rw-r--r-- 1 root root  47620501 Nov 12 06:20 debt_ts_pre_fips.csv
-rw-r--r-- 1 root root  49023971 Nov 13 19:58 debt_w_fips.csv
-rw-r--r-- 1 root root    104148 Nov 12 06:20 fips_st_co_02_07.csv
-rw-r--r-- 1 root root      7578 Nov 12 06:20 g_api_college.csv
-rw-r--r-- 1 root root    103193 Nov 12 06:20 g_api_rando.csv
-rw-r--r-- 1 root root   2874978 Nov 12 06:20 geocorr12.csv
-rw-r--r-- 1 root root     51068 Nov 13 18:50 state_coverage.csv
-rw-r--r-- 1 root root 107117255 Nov 13 18:50 tel_data_alt.csv
-rwxr-xr-x 1 root roo

Let's go ahead and read in the data.

In [228]:
#Read in data
data_in=pd.read_csv('../data/debt_out.csv')

#Capture issuer suffixes
issuers=['City, Town Vlg','Co-op Utility','College or Univ','County/Parish','Direct Issuer','District',\
         'Indian Tribe','Local Authority','State Authority','State/Province']
purposes=['Development','Education','Electric Power','Environmental Facilities','General Purpose','Healthcare',\
          'Housing','Public Facilities','Transportation','Utilities']

#Define replacement suffixes
issuers_new=['GEN_MUNI','COOP_UTIL','UNIV','CTY','DIRECT','DISTRICT','TRIBE','LOC_AUTH','ST_AUTH','STATE_GOV']
purposes_new=['DEV','EDUC','ELECTRIC','ENVIRON','GEN_PUR','HEALTH','HOUS','PUB_FAC','TRANSPORT','UTIL']

#Capture lists of old and new name pairings
goi=zip(['GO_'+var for var in issuers],['GO_'+var for var in issuers_new])
gop=zip(['GO_'+var for var in purposes],['GO_'+var for var in purposes_new])
rvi=zip(['RV_'+var for var in issuers],['RV_'+var for var in issuers_new])
rvp=zip(['RV_'+var for var in purposes],['RV_'+var for var in purposes_new])

#Build renaming dict
rename_dict=dict(goi+gop+rvi+rvp)

#Rename relevant variables
data_in=data_in.rename(columns=rename_dict)

print sorted(data_in.columns),'\n\n',data_in.info()

['ASMT_L', 'ASMT_L2', 'ASMT_L3', 'BOTH', 'CB_E', 'CB_E2', 'CB_E3', 'CB_E4', 'CB_G', 'CB_G2', 'CFDISC_L', 'CGEXP_L', 'CH_HS_UNT', 'CLEVY_L', 'CLEVY_L2', 'CLEVY_L3', 'CLEVY_L4', 'CRATE_L', 'CRATE_L2', 'CREVU_L', 'DENSITY', 'DIVERSITY', 'D_GEN_EXP', 'EDUC_SERV_EMP_PNFARM', 'EMP_RES', 'FFDISC_L', 'FIPS', 'FIPSST', 'FIPST_N', 'FOOD_SERV_EMP_PNFARM', 'GEN_REV', 'GEXP_L', 'GO', 'GO_COOP_UTIL', 'GO_CTY', 'GO_DEV', 'GO_DIRECT', 'GO_DISTRICT', 'GO_EDUC', 'GO_ELECTRIC', 'GO_ENVIRON', 'GO_GEN_MUNI', 'GO_GEN_PUR', 'GO_HEALTH', 'GO_HOUS', 'GO_LOC_AUTH', 'GO_PUB_FAC', 'GO_STATE_GOV', 'GO_ST_AUTH', 'GO_TRANSPORT', 'GO_TRIBE', 'GO_UNIV', 'GO_UTIL', 'GP_GEXP', 'GP_LEVY', 'GP_LMT', 'GP_RATE', 'GP_REVU', 'HOME_STEAD', 'HOME_STEAD2', 'HOME_STEAD3', 'HSG_UNITS', 'HSG_UNITS_ACS', 'HSLD_PERS', 'IGR_ST', 'LANDAREA', 'LEVY_L', 'LIMITS', 'MANU_EMP_PNFARM', 'MANU_RES', 'MDHOMEVAL', 'MED_INC', 'MFDISC_L', 'MFG_EMP', 'MGEXP_L', 'MGEXP_L2', 'MLEVY_L', 'MLEVY_L2', 'MLEVY_L3', 'MLEVY_L4', 'MRATE_L', 'MRATE_L2', 'MREVU

The set of variables in play appear in the table below:

**DEPENDENT VARIABLES**

Concept|Input Variables
-------|---------------
Per capita GO debt issued|*Variables beginning with GO* & `RES_POP`
Per capita revenue debt issued|*Variables beginning with RV* & `RES_POP`
Ratio of GO to revenue debt issued|*Variables beginning with GO or RV*

**INSTITUTIONAL VARIABLES**

Concept|Input Variables
-------|---------------
Any TEL|`LIMITS`
Non-binding TEL|`TYPE1`
Potentially binding TEL|`TYPE2`
Both `TYPE1` & `TYPE2`|`BOTH`
Years since `TYPE2` enacted|`TYPE2_y`
Overall property tax rate limit|`RATE_L`
Overall assessment limit|`SC_LMT`
Limit applied to general purpose gov|`GP_LMT`
Limit applied to school district|`SC_LMT`

*Note that all limits above can be interacted with primary county status (`PRIMARY`; see spatial table below), in which case we append an `i` to the variable name.*

**SCALE & SUPPLY MEASURES**

Concept|Input Variables
-------|---------------
Population|`RES_POP`
<span style="color:red">Population$^2$</span>|`RES_POP2`
Population density|`DENSITY`
Population growth rate|`POPGROW`
Household size|`PERS_HLD`
Pre-1940 housing stock|`PRE1940`

**DEMAND MEASURES**

Concept|Input Variables
-------|---------------
Population under 17|`PYOUNG`
Private school enrollment|`PVT_SCH`
Population over 65|`POP65`
Per capita income|`PCINC`
Povery rate|`POVERTY`
Average monthly Social Security payments (to recipients)|`PC_SSI`
Per capita income weighted by poverty rate|`DIVERSITY`

**ECONOMIC ACTIVITY**

Concept|Input Variables
-------|---------------
Employment to population ratio|`EMP_RESI`
Manufacturing employment to population ratio|`MANU_RES`
Retail employment to population ratio|`RETL_RES`
Service employment to population ratio|`SERV_RES`

**SPATIAL CHARACTERISTICS**

Concept|Input Variables
-------|---------------
Primary central county in 1974|`PRIMARY`
Co-central county in 1974|`CO_PRIM`
Urban fringe county in 1974|`FRINGE`

Let's grab these in category lists to make them more accessible.

In [229]:
#Capture dependent variables
debt_vars=['GO','RV']
go_vars={'iss':['GO_'+var for var in issuers_new],
         'pur':['GO_'+var for var in purposes_new]}
rv_vars={'iss':['RV_'+var for var in issuers_new],
         'pur':['RV_'+var for var in purposes_new]}

#Capture independnet vars
tel_vars={'types':['TYPE1','TYPE2','TYPE2_Y'],
          'either':['LIMITS','BOTH'],
          'hi_res':['RATE_L','ASMT_L','GP_LMT','SC_LMT']}
supply_vars=['RESPOP','DENSITY','POPGROWTH','HSLD_PERS','PRE1940']
demand_vars=['PYOUNG','PVT_SCH','POP65','PC_INC','POVERTY','PC_SSI','DIVERSITY']
economic_vars=['EMP_RES','MANU_RES','RETL_RES','SERV_RES']
spatial_vars=['PRIMARY','CO_PRIM','FRINGE']

#Capture all modeling variables in a single list
mod_vars=debt_vars+go_vars['iss']+go_vars['pur']+rv_vars['iss']+rv_vars['pur']+tel_vars['types']+\
         tel_vars['either']+tel_vars['hi_res']+supply_vars+demand_vars+economic_vars+spatial_vars
    
#For each model variable...
for var in mod_vars:
    #...tell me if it's not in the set
    if var not in data_in.columns:
        print "Is "+var+" in the data set??  Maaaan, we ain't found shit!"
        
#Capture model subset
data=data_in[[var for var in mod_vars if var in data_in.columns]+['Year','FIPS']]

#Generate populations squared
data['RESPOP2']=data['RESPOP']**2

data.head().T

Is PRIMARY in the data set??  Maaaan, we ain't found shit!
Is CO_PRIM in the data set??  Maaaan, we ain't found shit!
Is FRINGE in the data set??  Maaaan, we ain't found shit!


/home/choct155/analysis/Anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,0,1,2,3,4
GO,0.000000e+00,0.000000e+00,6.350000e+00,0.000000e+00,1.000000e+00
RV,7.979520e+02,1.625000e+00,2.333000e+01,4.000000e-01,1.425000e+00
GO_GEN_MUNI,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_COOP_UTIL,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_UNIV,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_CTY,0.000000e+00,0.000000e+00,6.350000e+00,0.000000e+00,1.000000e+00
GO_DIRECT,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_DISTRICT,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_TRIBE,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GO_LOC_AUTH,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


### Accounting for Inflation

The first thing we need to do is adjust all of the dollar figures for inflation.  The relevant variables are in the following list.

In [230]:
#Capture debt variables
debt_vars=[var for var in data.columns if var.startswith('GO')]+\
          [var for var in data.columns if var.startswith('RV')]

#Capture variables to be deflated
defl_vars=debt_vars+['PC_INC','PC_SSI','DIVERSITY']

We need to pull an index for deflation.  To be consistent with the descriptive analysis, we will use the PCE deflator from [FRED](https://research.stlouisfed.org/fred2/).  Specifically, we are using the chained PCE deflator ([PCECTPI](https://research.stlouisfed.org/fred2/)).  We can pull that directly with pandas FRED API and use it to deflate our data.

**UPDATE** We are converting to the state and local government implicit price deflator (`A829RD3A086NBEA`).  The mechanics are the same as with the PCE, so we are leaving the code undisturbed.  Note that the SLD increases faster than the PCE.

In [231]:
#Capture PCE
# pce=web.DataReader("PCECTPI","fred",'1/1/1980','1/1/2015').reset_index()
pce=web.DataReader("A829RD3A086NBEA","fred",'1/1/1980','1/1/2015').reset_index()

#Pull out year
pce['Year']=pce['DATE'].apply(lambda x: x.year)

#Set index
pce.set_index('Year',inplace=True)

#Drop date
pce.pop('DATE')

#Capture average by year
pce=pce.groupby(level='Year').mean()

#Calculate deflators
# pce['defl']=pce['PCECTPI'].apply(lambda x: pce['PCECTPI'].ix[2009]/x)
pce['defl']=pce['A829RD3A086NBEA'].apply(lambda x: pce['A829RD3A086NBEA'].ix[2009]/x)

#Map deflator into data via Year
data['defl']=data['Year'].map(pce['defl'])

print pce.head()

data[['Year','defl']].head(20)

      A829RD3A086NBEA      defl
Year                           
1980           32.583  3.069085
1981           35.824  2.791425
1982           38.012  2.630748
1983           39.700  2.518892
1984           41.407  2.415051


/home/choct155/analysis/Anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Year,defl
0,1984,2.415051
1,1984,2.415051
2,1984,2.415051
3,1984,2.415051
4,1984,2.415051
5,1984,2.415051
6,1984,2.415051
7,1984,2.415051
8,1984,2.415051
9,1984,2.415051


With the deflators in hand, we can loop through a copy of the data and generate deflated versions of the contents of `defl_vars`.

In [232]:
#Create a data copy to hold deflated values
data_d=data.copy(deep=True)

#For each var to be deflated...
for var in defl_vars:
    #....deflate the variable
    data_d[var]=data_d[var]*data_d['defl']

print data[['GO','RV','Year','defl']].head()    
print '\n',data_d[['GO','RV','Year','defl']].head()

     GO       RV  Year      defl
0  0.00  797.952  1984  2.415051
1  0.00    1.625  1984  2.415051
2  6.35   23.330  1984  2.415051
3  0.00    0.400  1984  2.415051
4  1.00    1.425  1984  2.415051

          GO           RV  Year      defl
0   0.000000  1927.094453  1984  2.415051
1   0.000000     3.924457  1984  2.415051
2  15.335571    56.343130  1984  2.415051
3   0.000000     0.966020  1984  2.415051
4   2.415051     3.441447  1984  2.415051


### Accounting for the Business Cycle

We also need to grab recessionary dates, but some assumptions are required since those are tracked on an monthly basis.  Our imperfect approach will be to treat an year with a majority of months in recession as being a recessionary year.  Since the annual average leaves us with the proportion of recessionary months, a recessionary year is any one in which the average exceeds 0.5.

In [233]:
#Capture recessionary dates
usrec=web.DataReader("USREC","fred",'1/1/1980','1/1/2015').reset_index()

#Pull out year
usrec['Year']=usrec['DATE'].apply(lambda x: x.year)

#Set index
usrec.set_index('Year',inplace=True)

#Drop date
usrec.pop('DATE')

#Capture average by year
usrec=usrec.groupby(level='Year').mean()

#Calculate binary recession variable
usrec['BIN_REC']=np.where(usrec['USREC']>=.5,1,0)

#Map bin_rec into data via Year
data_d['BIN_REC']=data_d['Year'].map(usrec['BIN_REC'])

print usrec.head()

print data_d[['Year','BIN_REC']].head()
print data_d[data_d['Year']==2001][['Year','BIN_REC']].head()

         USREC  BIN_REC
Year                   
1980  0.500000        1
1981  0.416667        0
1982  0.916667        1
1983  0.000000        0
1984  0.000000        0
   Year  BIN_REC
0  1984        0
1  1984        0
2  1984        0
3  1984        0
4  1984        0
       Year  BIN_REC
31774  2001        1
31775  2001        1
31776  2001        1
31777  2001        1
31778  2001        1


### Mapping in State Labels

Just for facilitating the reading of results, state labels would be useful.  Here is a mapping from that FIPS codes.

In [234]:
#Capture state FIPS codes
data_d['FIPSST']=data_d['FIPS'].apply(lambda x: str(x).zfill(5)[:2])

#Map FIPS to state
fips_st_map={'01': 'AL',
             '02': 'AK',
             '04': 'AZ',
             '05': 'AR',
             '06': 'CA',
             '08': 'CO',
             '09': 'CT',
             '10': 'DE',
             '11': 'DC',
             '12': 'FL',
             '13': 'GA',
             '15': 'HI',
             '16': 'ID',
             '17': 'IL',
             '18': 'IN',
             '19': 'IA',
             '20': 'KS',
             '21': 'KY',
             '22': 'LA',
             '23': 'ME',
             '24': 'MD',
             '25': 'MA',
             '26': 'MI',
             '27': 'MN',
             '28': 'MS',
             '29': 'MO',
             '30': 'MT',
             '31': 'NE',
             '32': 'NV',
             '33': 'NH',
             '34': 'NJ',
             '35': 'NM',
             '36': 'NY',
             '37': 'NC',
             '38': 'ND',
             '39': 'OH',
             '40': 'OK',
             '41': 'OR',
             '42': 'PA',
             '44': 'RI',
             '45': 'SC',
             '46': 'SD',
             '47': 'TN',
             '48': 'TX',
             '49': 'UT',
             '50': 'VT',
             '51': 'VA',
             '53': 'WA',
             '54': 'WV',
             '55': 'WI',
             '56': 'WY'}

#Map in state labels
data_d['STATE']=data_d['FIPSST'].map(fips_st_map)

It would also be useful to cluster by BEA region, so we need to map those in as well.

In [235]:
#Capture BEA regions
bea_reg_map={'NENG':['CT','ME','MA','NH','RI','VT'],
             'MEST':['DE','DC','MD','NJ','NY','PA'],
             'GLAK':['IL','IN','MI','OH','WI'],
             'PLNS':['IA','KS','MN','MO','NE','ND','SD'],
             'SEST':['AL','AR','FL','GA','KY','LA','MS','NC','SC','TN','VA','WV'],
             'SWST':['AZ','NM','OK','TX'],
             'RKMT':['CO','ID','MT','UT','WY'],
             'FWST':['AK','CA','HI','NV','OR','WA']}

#Provide integer labels for each region
bea_reg_ints={'NENG':1,
              'MEST':2,
              'GLAK':3,
              'PLNS':4,
              'SEST':5,
              'SWST':6,
              'RKMT':7,
              'FWST':8}

#Create container for tuples (st to reg)
st_reg_tups=[]

#For each region...
for reg in bea_reg_map.keys():
    #...and for each state in the region...
    for st in bea_reg_map[reg]:
        #...throw the st-reg pair in st_reg_tups
        st_reg_tups.append((st,reg))
        
#Capture reverse BEA region dict
bea_st_map=dict(st_reg_tups)

#Define BEA region variable
data_d['BEA']=data_d['STATE'].map(bea_st_map)

It turns out we have issues in here from American Samoa, Guam, Puerto Rico, and the Virgin Islands.  Since they do not have BEA regions, we will be dropping them.  There are only 85 records of this type.

In [236]:
#Subset to states
data_d=data_d[~data_d['FIPSST'].isin(['60', '72', '66', '78'])]

#Define integer version of BEA region
data_d['BEA_INT']=data_d['BEA'].map(bea_reg_ints)

data_d[['FIPSST','FIPS','STATE','BEA']].head()

,FIPSST,FIPS,STATE,BEA
0,01,1000,AL,SEST
1,01,1001,AL,SEST
2,01,1003,AL,SEST
3,01,1007,AL,SEST
4,01,1021,AL,SEST


In [237]:
data_d[data_d['BEA'].isnull()][['FIPSST','FIPS','STATE','BEA']]

,FIPSST,FIPS,STATE,BEA


## Model Design

### Dependent Variables

Now that we have our deflated data, we need to generate a few dependent variables:

1. Per Capita GO Debt Issued
2. Per Capita Revenue Debt Issued
3. Proportion of All Debt Issued that is GO

Note also that while we have the totals for all of these items, we also have GO and revenue debt by issuer type and debt purpose.  All of these could serve as dependent variables, so we should generate them systematically.  The first step would be capturing all per capita measures.

In [238]:
#Create container for per capita debt variables
pc_debt_vars=[]

#For each debt variable...
for var in debt_vars:
    #...generate per capita versions...
    data_d[var+'_PC']=data_d[var]/data_d['RESPOP']
    #...and store the variable name
    pc_debt_vars.append(var+'_PC')

print pc_debt_vars

['GO_PC', 'GO_GEN_MUNI_PC', 'GO_COOP_UTIL_PC', 'GO_UNIV_PC', 'GO_CTY_PC', 'GO_DIRECT_PC', 'GO_DISTRICT_PC', 'GO_TRIBE_PC', 'GO_LOC_AUTH_PC', 'GO_ST_AUTH_PC', 'GO_STATE_GOV_PC', 'GO_DEV_PC', 'GO_EDUC_PC', 'GO_ELECTRIC_PC', 'GO_ENVIRON_PC', 'GO_GEN_PUR_PC', 'GO_HEALTH_PC', 'GO_HOUS_PC', 'GO_PUB_FAC_PC', 'GO_TRANSPORT_PC', 'GO_UTIL_PC', 'RV_PC', 'RV_GEN_MUNI_PC', 'RV_COOP_UTIL_PC', 'RV_UNIV_PC', 'RV_CTY_PC', 'RV_DIRECT_PC', 'RV_DISTRICT_PC', 'RV_TRIBE_PC', 'RV_LOC_AUTH_PC', 'RV_ST_AUTH_PC', 'RV_STATE_GOV_PC', 'RV_DEV_PC', 'RV_EDUC_PC', 'RV_ELECTRIC_PC', 'RV_ENVIRON_PC', 'RV_GEN_PUR_PC', 'RV_HEALTH_PC', 'RV_HOUS_PC', 'RV_PUB_FAC_PC', 'RV_TRANSPORT_PC', 'RV_UTIL_PC']


The set of proportional dependents can also be generated systematically.

In [239]:
#Capture absolute debt pairs
debt_var_pairs=zip(debt_vars[:len(debt_vars)/2],debt_vars[len(debt_vars)/2:])

#Create a container for proportional debt variables
prop_debt_vars=[]

#For each pair...
for dp in debt_var_pairs:
    #...generate the GO proportion of total debt (in that area)...
    data_d[dp[0]+'_PROP']=data_d[dp[0]]/(data_d[dp[0]]+data_d[dp[1]])
    #...and capture the variable name
    prop_debt_vars.append(dp[0]+'_PROP')
    
print prop_debt_vars

['GO_PROP', 'GO_GEN_MUNI_PROP', 'GO_COOP_UTIL_PROP', 'GO_UNIV_PROP', 'GO_CTY_PROP', 'GO_DIRECT_PROP', 'GO_DISTRICT_PROP', 'GO_TRIBE_PROP', 'GO_LOC_AUTH_PROP', 'GO_ST_AUTH_PROP', 'GO_STATE_GOV_PROP', 'GO_DEV_PROP', 'GO_EDUC_PROP', 'GO_ELECTRIC_PROP', 'GO_ENVIRON_PROP', 'GO_GEN_PUR_PROP', 'GO_HEALTH_PROP', 'GO_HOUS_PROP', 'GO_PUB_FAC_PROP', 'GO_TRANSPORT_PROP', 'GO_UTIL_PROP']


### Debt Proportions

It may also prove useful to capture the proportions of debt by issuer type or purpose.  The idea here would be to explore the impact of debt classifications on the issuance of debt.  (In other words, we are talking about these proportions going on the independent side of the equation.)  Proportionality allows us to separate these regressors from the absolute debt figures on the dependent side.

For this purpose, we will actually focus on total debt to limit model complexity.

In [240]:
#Calculate total debt
data_d['TOT_DEBT']=data_d['GO']+data_d['RV']

#Calculate total debt per capita
data_d['TOT_DEBT_PC']=data_d['TOT_DEBT']/data_d['RESPOP']

#Create container for total debt variables
tot_debt_vars=['TOT_DEBT']

#For each of the remaining debt pairs...
for dp in debt_var_pairs[1:]:
    #...calculate the total...
    data_d['TOT'+dp[0][2:]]=data_d[dp[0]]+data_d[dp[1]]
    #...and capture the var name
    tot_debt_vars.append('TOT'+dp[0][2:])
    
#Create a container for proportions of total debt
prop_of_tot_debt_vars=[]

#For each of the total debt subsets...
for td in tot_debt_vars[1:]:
    #...calculate the proportion of total debt...
    data_d['TPROP'+td[3:]]=data_d[td]/data_d['TOT_DEBT']
    #...and capture the name
    prop_of_tot_debt_vars.append('TPROP'+td[3:])
    
#Split out type from purpose
tprop_vars={'ISSUER':prop_of_tot_debt_vars[:10],
            'PURPOSE':prop_of_tot_debt_vars[10:]}
tprop_vars

{'ISSUER': ['TPROP_GEN_MUNI',
  'TPROP_COOP_UTIL',
  'TPROP_UNIV',
  'TPROP_CTY',
  'TPROP_DIRECT',
  'TPROP_DISTRICT',
  'TPROP_TRIBE',
  'TPROP_LOC_AUTH',
  'TPROP_ST_AUTH',
  'TPROP_STATE_GOV'],
 'PURPOSE': ['TPROP_DEV',
  'TPROP_EDUC',
  'TPROP_ELECTRIC',
  'TPROP_ENVIRON',
  'TPROP_GEN_PUR',
  'TPROP_HEALTH',
  'TPROP_HOUS',
  'TPROP_PUB_FAC',
  'TPROP_TRANSPORT',
  'TPROP_UTIL']}

### Indepenent Variables

There are a few regressor lists we should construct, which vary largely with respect to how TELs are represented.

In [241]:
#Capture lists of regressors
reg1=['TYPE1','TYPE2','TYPE2_Y','RESPOP','RESPOP2','DENSITY','POPGROWTH','HSLD_PERS','PRE1940',\
        'PYOUNG','PVT_SCH','POP65','PC_INC','POVERTY','PC_SSI','DIVERSITY','EMP_RES','MANU_RES',\
        'RETL_RES','SERV_RES','BIN_REC']
reg2=['LIMITS','BOTH']+model1[3:]
reg3=['RATE_L','ASMT_L','GP_LMT','SC_LMT']+model1[3:]

#Capture in dict
reg_dict={'TYPE':reg1,
          'AGG':reg2,
          'HIRES':reg3}

### Fixed Effects

We have four fixed effect options which will be included in this analysis: `pooled`, `year fixed effects`, `state fixed effects`, and `both`.  These can be appended to each specification via simple extensions.  These extensions can be captured in a dictionary for easy access.

In [242]:
fe_dict={'POOLED':[],
         'YEAR':['C(Year)'],
         'STATE':['C(FIPSST)'],
         'BOTH':['C(Year)+C(FIPSST)']}

### Specification Builds

We now have host of moving parts on the specification side of the equation.  There are 63 possible dependent variables for the debt type breakouts (`GO` vs `RV`) and one total debt dependent for use in two models evaluating the debt composition impact.  Couple this with three regressor sets and four fixed effect options, and we have a whole mess of specifications (780).  

An orderly collection of formulas would be most useful, to say the least.  It perhaps goes without saying here, but not all of these models will receive the same level of scrutiny.  We will focus on a subset in any detailed discussion of results.  However, running all of these models gives us a chance to test variance in our TEL measures under a large variety of specifications.  Each group of TEL variables will have 260 estimates associated with it.

We will house our specifications in a hierarchical dictionary.  The first level keys will capture four main groups:

1. **Per Capita GO models** (`PC_GO`) are defined by their dependent variables (all per capita GO variables) at the next level down.  Only the simple regressor lists (without debt composition by issuer type or purpose) will be used.
2. **Per Capita RV models** (`PC_RV`) are defined by their dependent variables (all per capita RV variables) at the next level down.  Only the simple regressor lists (without debt composition by issuer type or purpose) will be used.
3. **Proportional Models** (`PROP`) both use total debt issued as the dependent.  The simple regressor lists are augmented by debt composition.  Whether this composition is split by issuer type (`ISSUER`) or debt purpose (`PURPOSE`) defines these models at the next level.

The second level being captured by the definitions within each group of the first level, the third level rotates across each regressor list captured in `reg_dict`.  The fourth level captures the fixed effect combinations.

The first level groups differ strongly due to the proportional group, so we will build each group separetely, and then manually combine them in the master dictionary.

In [243]:
## DEFINE FUNCTION TO CAPTURE PER CAPITA SPECS ##
def pc_spec_dict(debt_type):
    #Create dictionary to hold outgoing specs
    out_dict={}
    #For each per capita dependent...
    for dep in [var for var in pc_debt_vars if var.startswith(debt_type)]:
        #...create a temp dict to hold specs for all three regressor groups...
        tmp_reg_dict={}
        #...and for each regressor set...
        for rd in reg_dict.keys():
            #...create a temp dict to hold specs for all four fixed effect types...
            tmp_spec_dict={}
            #...and for each FE type...
            for fe in fe_dict.keys():
                #...build the spec and throw it in tmp_spec_dict...
                tmp_spec_dict.update({fe:dep+'~'+'+'.join(reg_dict[rd]+fe_dict[fe])})
            #...once tmp_spec_dict is full, throw it in tmp_reg_dict...
            tmp_reg_dict.update({rd:tmp_spec_dict})
        #...and once tmp_reg_dict is full, throw it in out_dict
        if dep[3:-3]=='':
            out_dict.update({'Total':tmp_reg_dict})
        else:
            out_dict.update({dep[3:-3]:tmp_reg_dict})
    return out_dict

## CAPTURE DICTIONARY FOR PER CAPITA GO SPECS ##
go_spec_dict=pc_spec_dict('GO')

## CAPTURE DICTIONARY FOR PER CAPITA GO SPECS ##
rv_spec_dict=pc_spec_dict('RV')

## CAPTURE DICTIONARY FOR PROPORTIONAL SPECS ##

#Create dictionary to hold prop specs
prop_dict={}

#For each prop key...
for key in ['ISSUER','PURPOSE']:
    #...create a temp dict to hold specs for all three regressor groups...
    tmp_reg_dict={}
    #...and for each regressor set...
    for rd in reg_dict.keys():
        #...create a temp dict to hold specs for all four fixed effect types...
        tmp_spec_dict={}
        #...and for each FE type...
        for fe in fe_dict.keys():
            #...build the spec and throw it in tmp_spec_dict...
            tmp_spec_dict.update({fe:'TOT_DEBT_PC'+'~'+'+'.join(reg_dict[rd]+tprop_vars[key]+fe_dict[fe])})
        #...once tmp_spec_dict is full, throw it in tmp_reg_dict...
        tmp_reg_dict.update({rd:tmp_spec_dict})
    #...and once tmp_reg_dict is full, throw it in prop_dict
    prop_dict.update({key:tmp_reg_dict})
    
## CAPTURE FIRST LEVEL DICTS IN SPECIFICATIONS DICT ##
spec_dict={'PC_GO':go_spec_dict,
           'PC_RV':rv_spec_dict,
           'PROP':prop_dict}

Just to provide a sense of how this works, let's say we want to regress revenue debt per capita for housing on TELs split by type with year fixed effects, we can call that spec with the following:

In [244]:
spec_dict['PC_RV']['HOUS']['TYPE']['YEAR']

'RV_HOUS_PC~TYPE1+TYPE2+TYPE2_Y+RESPOP+RESPOP2+DENSITY+POPGROWTH+HSLD_PERS+PRE1940+PYOUNG+PVT_SCH+POP65+PC_INC+POVERTY+PC_SSI+DIVERSITY+EMP_RES+MANU_RES+RETL_RES+SERV_RES+BIN_REC+C(Year)'

If I wanted to evaluate the impact of aggregate TEL measures and debt composition by issuer type on total debt per capita, using state fixed effects, this call would work:

In [245]:
spec_dict['PROP']['ISSUER']['AGG']['STATE']

'TOT_DEBT_PC~LIMITS+BOTH+RESPOP+RESPOP2+DENSITY+POPGROWTH+HSLD_PERS+PRE1940+PYOUNG+PVT_SCH+POP65+PC_INC+POVERTY+PC_SSI+DIVERSITY+EMP_RES+MANU_RES+RETL_RES+SERV_RES+TPROP_GEN_MUNI+TPROP_COOP_UTIL+TPROP_UNIV+TPROP_CTY+TPROP_DIRECT+TPROP_DISTRICT+TPROP_TRIBE+TPROP_LOC_AUTH+TPROP_ST_AUTH+TPROP_STATE_GOV+C(FIPSST)'

## Estimation

Now we are in a position to estimate some models.  We will first estimate some top line models, and then move into the analysis of all specs.  The top line models are as follows:

1. Total GO debt per capita on TELs by type with year and state fixed effects;
2. Total RV debt per capita on TELs by type with year and state fixed effects;
3. Total debt per capita on TELs by type and debt composition by issuer type, with year and state fixed effects;
4. Total debt per capita on TELs by type and debt composition by purpose, with year and state fixed effects.

In [246]:
smf.ols(formula=topline_specs['GO'],data=data_d)

In [247]:
#Capture topline specs
topline_specs={'GO':spec_dict['PC_GO']['Total']['TYPE']['BOTH'],
               'RV':spec_dict['PC_RV']['Total']['TYPE']['BOTH'],
               'ISSUER':spec_dict['PROP']['ISSUER']['TYPE']['BOTH'],
               'PURPOSE':spec_dict['PROP']['PURPOSE']['TYPE']['BOTH']}

#Estimate each model
# topline_mods={'GO':smf.ols(formula=topline_specs['GO'],data=data_d).fit(cov_type='cluster',
#                                                                        cov_kwds={'groups':data_d['BEA_INT']})}
topline_mods={'GO':smf.ols(formula=topline_specs['GO'],data=data_d).fit(cov_type='HC0'),
              'RV':smf.ols(formula=topline_specs['RV'],data=data_d).fit(cov_type='HC0'),
              'ISSUER':smf.ols(formula=topline_specs['ISSUER'],data=data_d).fit(cov_type='HC0'),
              'PURPOSE':smf.ols(formula=topline_specs['PURPOSE'],data=data_d).fit(cov_type='HC0')}

topline_mods['GO'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  GO_PC   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     317.1
Date:                Sat, 14 Nov 2015   Prob (F-statistic):               0.00
Time:                        14:58:09   Log-Likelihood:             3.0072e+05
No. Observations:               51499   AIC:                        -6.014e+05
Df Residuals:                   51492   BIC:                        -6.014e+05
Df Model:                           6                                         
Covariance Type:                  HC0                                         
===================================================================================
                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept       -6.527e-05   5.88e-05     -1.111      0.267        -0.000  4.99e-05
C(Year)[T.1985]  8.523e-05   7.64e-05      1.116      0.264     -6.45e-05     0.000
C(Year)[T.1986] -5.973e-05   2.19e-05     -2.726      0.006        -0.000 -1.68e-05
C(Year)[T.1987] -4.131e-05   2.08e-05     -1.990      0.047      -8.2e-05 -6.29e-07
C(Year)[T.1988] -9.422e-05   1.74e-05     -5.421      0.000        -0.000 -6.02e-05
C(Year)[T.1989]  -4.44e-05   1.66e-05     -2.674      0.007     -7.69e-05 -1.19e-05
C(Year)[T.1990] -4.778e-05    1.7e-05     -2.807      0.005     -8.11e-05 -1.44e-05
C(Year)[T.1991] -2.423e-06   1.72e-05     -0.141      0.888     -3.61e-05  3.12e-05
C(Year)[T.1992]  3.634e-05   2.05e-05      1.776      0.076     -3.75e-06  7.64e-05
C(Year)[T.1993]  7.683e-05   2.34e-05      3.281      0.001      3.09e-05     0.000
C(Year)[T.1994] -7.851e-06   1.84e-05     -0.427      0.669     -4.39e-05  2.82e-05
C(Year)[T.1995] -4.901e-05   1.87e-05     -2.623      0.009     -8.56e-05 -1.24e-05
C(Year)[T.1996]  5.633e-06   2.09e-05      0.270      0.787     -3.53e-05  4.66e-05
C(Year)[T.1997] -1.603e-05   1.96e-05     -0.816      0.414     -5.45e-05  2.25e-05
C(Year)[T.1998]     0.0001   2.69e-05      4.260      0.000      6.18e-05     0.000
C(Year)[T.1999] -8.999e-06   1.99e-05     -0.452      0.652      -4.8e-05  3.01e-05
C(Year)[T.2000] -5.118e-06      2e-05     -0.256      0.798     -4.43e-05  3.41e-05
C(Year)[T.2001]  5.547e-05   1.11e-05      5.005      0.000      3.37e-05  7.72e-05
C(Year)[T.2002]  9.673e-05   2.78e-05      3.475      0.001      4.22e-05     0.000
C(Year)[T.2003]  9.684e-05   2.31e-05      4.193      0.000      5.16e-05     0.000
C(Year)[T.2004]  5.397e-05    2.1e-05      2.569      0.010      1.28e-05  9.52e-05
C(Year)[T.2005]  4.004e-05   2.13e-05      1.880      0.060     -1.69e-06  8.18e-05
C(Year)[T.2006] -3.924e-05   1.94e-05     -2.018      0.044     -7.74e-05 -1.12e-06
C(Year)[T.2007]  2.116e-05   3.46e-05      0.612      0.541     -4.66e-05   8.9e-05
C(Year)[T.2008] -3.151e-05   1.04e-05     -3.031      0.002     -5.19e-05 -1.11e-05
C(Year)[T.2009]  -2.03e-05   1.01e-05     -2.017      0.044        -4e-05 -5.75e-07
C(Year)[T.2010]  1.356e-05   2.13e-05      0.637      0.524     -2.82e-05  5.53e-05
C(Year)[T.2011] -1.115e-11   1.17e-12     -9.492      0.000     -1.34e-11 -8.85e-12
C(Year)[T.2012]  1.341e-11   1.41e-12      9.492      0.000      1.06e-11  1.62e-11
C(Year)[T.2013]  6.963e-12   7.34e-13      9.492      0.000      5.53e-12   8.4e-12
C(Year)[T.2014]  2.727e-11   2.87e-12      9.492      0.000      2.16e-11  3.29e-11
C(Year)[T.2015]  3.767e-11   3.97e-12      9.492      0.000      2.99e-11  4.54e-11
C(FIPSST)[T.02]  8.337e-05   3.14e-05      2.653      0.008      2.18e-05     0.000
C(FIPSST)[T.04]    -0.0001   2.62e-05     -5.037      0.000

In [248]:
topline_mods['RV'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  RV_PC   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     18.99
Date:                Sat, 14 Nov 2015   Prob (F-statistic):               0.00
Time:                        14:58:10   Log-Likelihood:             2.5262e+05
No. Observations:               51499   AIC:                        -5.052e+05
Df Residuals:                   51492   BIC:                        -5.052e+05
Df Model:                           6                                         
Covariance Type:                  HC0                                         
===================================================================================
                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept          -0.0001   9.61e-05     -1.206      0.228        -0.000  7.25e-05
C(Year)[T.1985]     0.0007      0.000      4.059      0.000         0.000     0.001
C(Year)[T.1986]    -0.0003   6.97e-05     -3.960      0.000        -0.000    -0.000
C(Year)[T.1987]    -0.0003    8.1e-05     -4.209      0.000        -0.000    -0.000
C(Year)[T.1988]    -0.0004   6.05e-05     -5.931      0.000        -0.000    -0.000
C(Year)[T.1989]    -0.0004   6.06e-05     -6.065      0.000        -0.000    -0.000
C(Year)[T.1990]    -0.0004   5.84e-05     -6.256      0.000        -0.000    -0.000
C(Year)[T.1991]    -0.0003   6.61e-05     -4.758      0.000        -0.000    -0.000
C(Year)[T.1992]    -0.0002   6.06e-05     -3.955      0.000        -0.000    -0.000
C(Year)[T.1993]    -0.0002   7.66e-05     -2.165      0.030        -0.000 -1.57e-05
C(Year)[T.1994]    -0.0003   6.19e-05     -5.322      0.000        -0.000    -0.000
C(Year)[T.1995]    -0.0003   5.66e-05     -5.692      0.000        -0.000    -0.000
C(Year)[T.1996]    -0.0004   5.99e-05     -5.858      0.000        -0.000    -0.000
C(Year)[T.1997]    -0.0003   5.39e-05     -6.226      0.000        -0.000    -0.000
C(Year)[T.1998]    -0.0002   6.32e-05     -3.758      0.000        -0.000    -0.000
C(Year)[T.1999]    -0.0003   5.86e-05     -5.524      0.000        -0.000    -0.000
C(Year)[T.2000]    -0.0004   5.51e-05     -6.689      0.000        -0.000    -0.000
C(Year)[T.2001] -3.746e-05   3.26e-05     -1.149      0.251        -0.000  2.65e-05
C(Year)[T.2002]    -0.0003   5.85e-05     -5.437      0.000        -0.000    -0.000
C(Year)[T.2003]    -0.0003   6.16e-05     -5.504      0.000        -0.000    -0.000
C(Year)[T.2004]    -0.0003   5.55e-05     -6.190      0.000        -0.000    -0.000
C(Year)[T.2005]    -0.0003   5.75e-05     -5.485      0.000        -0.000    -0.000
C(Year)[T.2006]    -0.0004   5.49e-05     -6.573      0.000        -0.000    -0.000
C(Year)[T.2007]    -0.0003   6.02e-05     -5.589      0.000        -0.000    -0.000
C(Year)[T.2008]    -0.0001   2.63e-05     -4.117      0.000        -0.000 -5.67e-05
C(Year)[T.2009]    -0.0001   2.28e-05     -5.967      0.000        -0.000 -9.13e-05
C(Year)[T.2010]    -0.0003   6.03e-05     -5.710      0.000        -0.000    -0.000
C(Year)[T.2011] -3.711e-12   2.67e-12     -1.389      0.165     -8.95e-12  1.53e-12
C(Year)[T.2012]  4.463e-12   3.21e-12      1.389      0.165     -1.83e-12  1.08e-11
C(Year)[T.2013]  2.318e-12   1.67e-12      1.389      0.165     -9.53e-13  5.59e-12
C(Year)[T.2014]  9.078e-12   6.54e-12      1.389      0.165     -3.73e-12  2.19e-11
C(Year)[T.2015]  1.254e-11   9.03e-12      1.389      0.165     -5.15e-12  3.02e-11
C(FIPSST)[T.02]    -0.0006   8.29e-05     -7.094      0.000        -0.001    -0.000
C(FIPSST)[T.04]    -0.0001   8.65e-05     -1.385      0.166

In [249]:
topline_mods['ISSUER'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            TOT_DEBT_PC   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     232.7
Date:                Sat, 14 Nov 2015   Prob (F-statistic):               0.00
Time:                        14:58:10   Log-Likelihood:             2.4760e+05
No. Observations:               51499   AIC:                        -4.952e+05
Df Residuals:                   51492   BIC:                        -4.951e+05
Df Model:                           6                                         
Covariance Type:                  HC0                                         
===================================================================================
                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept        2.789e-05      0.000      0.241      0.809        -0.000     0.000
C(Year)[T.1985]     0.0008      0.000      4.151      0.000         0.000     0.001
C(Year)[T.1986]    -0.0002   7.48e-05     -3.195      0.001        -0.000 -9.25e-05
C(Year)[T.1987]    -0.0003   7.88e-05     -3.471      0.001        -0.000    -0.000
C(Year)[T.1988]    -0.0003   6.49e-05     -5.383      0.000        -0.000    -0.000
C(Year)[T.1989]    -0.0003   6.16e-05     -5.237      0.000        -0.000    -0.000
C(Year)[T.1990]    -0.0003   6.33e-05     -4.862      0.000        -0.000    -0.000
C(Year)[T.1991]    -0.0003   7.03e-05     -3.853      0.000        -0.000    -0.000
C(Year)[T.1992] -7.392e-05   6.32e-05     -1.170      0.242        -0.000     5e-05
C(Year)[T.1993]     0.0002   8.63e-05      1.855      0.064     -9.06e-06     0.000
C(Year)[T.1994]    -0.0003   6.34e-05     -4.058      0.000        -0.000    -0.000
C(Year)[T.1995]    -0.0002   6.05e-05     -3.792      0.000        -0.000    -0.000
C(Year)[T.1996]    -0.0002   6.38e-05     -3.792      0.000        -0.000    -0.000
C(Year)[T.1997]    -0.0002    5.8e-05     -3.636      0.000        -0.000 -9.73e-05
C(Year)[T.1998]    -0.0001   6.96e-05     -1.635      0.102        -0.000  2.26e-05
C(Year)[T.1999]    -0.0001   5.94e-05     -2.489      0.013        -0.000 -3.14e-05
C(Year)[T.2000]    -0.0003   5.81e-05     -5.128      0.000        -0.000    -0.000
C(Year)[T.2001]  7.811e-05   3.49e-05      2.237      0.025      9.66e-06     0.000
C(Year)[T.2002]  -3.97e-05    6.2e-05     -0.640      0.522        -0.000  8.19e-05
C(Year)[T.2003]    -0.0001   6.25e-05     -1.654      0.098        -0.000  1.91e-05
C(Year)[T.2004]    -0.0002   5.89e-05     -3.069      0.002        -0.000 -6.53e-05
C(Year)[T.2005]    -0.0001   6.18e-05     -1.816      0.069        -0.000   8.9e-06
C(Year)[T.2006]    -0.0003   5.62e-05     -4.694      0.000        -0.000    -0.000
C(Year)[T.2007]    -0.0002   6.83e-05     -3.167      0.002        -0.000 -8.24e-05
C(Year)[T.2008]    -0.0001   2.85e-05     -4.107      0.000        -0.000 -6.12e-05
C(Year)[T.2009]    -0.0001   2.68e-05     -4.795      0.000        -0.000 -7.59e-05
C(Year)[T.2010]    -0.0002    6.5e-05     -3.628      0.000        -0.000    -0.000
C(Year)[T.2011]  4.012e-19   3.02e-19      1.328      0.184     -1.91e-19  9.93e-19
C(Year)[T.2012] -1.421e-18    4.2e-19     -3.386      0.001     -2.24e-18 -5.98e-19
C(Year)[T.2013]  -2.27e-19   1.89e-19     -1.199      0.230     -5.98e-19  1.44e-19
C(Year)[T.2014] -7.147e-19      4e-19     -1.787      0.074      -1.5e-18  6.92e-20
C(Year)[T.2015] -4.774e-19   2.02e-19     -2.362      0.018     -8.74e-19 -8.13e-20
C(FIPSST)[T.02]    -0.0002   5.64e-05     -4.177      0.000        -0.000    -0.000
C(FIPSST)[T.04]    -0.0004   7.83e-05     -4.483      0.000

In [250]:
topline_mods['PURPOSE'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            TOT_DEBT_PC   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     242.6
Date:                Sat, 14 Nov 2015   Prob (F-statistic):               0.00
Time:                        14:58:10   Log-Likelihood:             2.4904e+05
No. Observations:               51499   AIC:                        -4.981e+05
Df Residuals:                   51492   BIC:                        -4.980e+05
Df Model:                           6                                         
Covariance Type:                  HC0                                         
===================================================================================
                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           0.0003      0.000      1.724      0.085     -4.39e-05     0.001
C(Year)[T.1985]     0.0006      0.000      3.107      0.002         0.000     0.001
C(Year)[T.1986]    -0.0002    7.5e-05     -2.991      0.003        -0.000 -7.73e-05
C(Year)[T.1987]    -0.0003   7.58e-05     -3.721      0.000        -0.000    -0.000
C(Year)[T.1988]    -0.0003    6.4e-05     -4.816      0.000        -0.000    -0.000
C(Year)[T.1989]    -0.0002   5.95e-05     -3.682      0.000        -0.000    -0.000
C(Year)[T.1990]    -0.0003   6.34e-05     -4.764      0.000        -0.000    -0.000
C(Year)[T.1991]    -0.0002   7.01e-05     -3.290      0.001        -0.000 -9.33e-05
C(Year)[T.1992] -6.143e-05   6.57e-05     -0.934      0.350        -0.000  6.74e-05
C(Year)[T.1993]     0.0001   8.39e-05      1.335      0.182     -5.24e-05     0.000
C(Year)[T.1994]    -0.0002   6.49e-05     -3.609      0.000        -0.000    -0.000
C(Year)[T.1995]    -0.0003   6.46e-05     -4.032      0.000        -0.000    -0.000
C(Year)[T.1996]    -0.0002   6.76e-05     -2.426      0.015        -0.000 -3.15e-05
C(Year)[T.1997]    -0.0002   6.17e-05     -2.531      0.011        -0.000 -3.52e-05
C(Year)[T.1998]    -0.0001   7.53e-05     -1.369      0.171        -0.000  4.46e-05
C(Year)[T.1999]    -0.0001   6.35e-05     -1.741      0.082        -0.000  1.39e-05
C(Year)[T.2000]    -0.0002   6.21e-05     -3.649      0.000        -0.000    -0.000
C(Year)[T.2001]  4.167e-05   3.47e-05      1.201      0.230     -2.63e-05     0.000
C(Year)[T.2002]  4.346e-05   6.64e-05      0.655      0.512     -8.66e-05     0.000
C(Year)[T.2003]  5.453e-05   6.23e-05      0.876      0.381     -6.75e-05     0.000
C(Year)[T.2004] -7.846e-05   6.39e-05     -1.228      0.219        -0.000  4.68e-05
C(Year)[T.2005] -2.369e-05    6.7e-05     -0.354      0.724        -0.000     0.000
C(Year)[T.2006]    -0.0002   6.27e-05     -2.800      0.005        -0.000 -5.27e-05
C(Year)[T.2007]    -0.0001   7.42e-05     -1.688      0.091        -0.000  2.02e-05
C(Year)[T.2008] -4.093e-05   2.78e-05     -1.473      0.141     -9.54e-05  1.35e-05
C(Year)[T.2009] -9.826e-05    2.7e-05     -3.633      0.000        -0.000 -4.53e-05
C(Year)[T.2010]    -0.0002   7.15e-05     -2.318      0.020        -0.000 -2.56e-05
C(Year)[T.2011]  7.875e-14   3.58e-14      2.199      0.028      8.56e-15  1.49e-13
C(Year)[T.2012] -1.674e-13   7.61e-14     -2.199      0.028     -3.17e-13 -1.82e-14
C(Year)[T.2013] -3.534e-14   1.61e-14     -2.199      0.028     -6.68e-14 -3.84e-15
C(Year)[T.2014] -1.095e-13   4.98e-14     -2.199      0.028     -2.07e-13 -1.19e-14
C(Year)[T.2015]   2.66e-14   1.21e-14      2.199      0.028      2.89e-15  5.03e-14
C(FIPSST)[T.02] -8.596e-05   5.93e-05     -1.449      0.147        -0.000  3.03e-05
C(FIPSST)[T.04]    -0.0001   8.69e-05     -1.256      0.209

Add in years

dummy for recessions

cluster by BEA region

implicit price deflator for state and local government services

try total volume as dependent and proportions

interest rates